In [1]:
import pandas as pd
import numpy as np
from random import randrange, uniform
from sklearn import linear_model

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:1085: UserWarning: Duplicate key in file "/Users/Pani/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


height has been deprecated.



In [2]:
import os
files = os.listdir('we_data/')
print(files)

['.DS_Store', 'Group_xx.csv', 'test.csv', 'train.csv', 'validation.csv']


In [3]:

train = pd.read_csv('we_data/train.csv')
test = pd.read_csv('we_data/test.csv')
validation = pd.read_csv('we_data/validation.csv')

In [4]:
# list(train)

# #Number of training points
# num_train_data = train.count()
# print("Number of training data ", num_train_data)

# #Number of features
# num_fut = len(list(train))
# print("number of features", num_fut)


# # ad_group = train.groupby('advertiser')
# # ad_group['advertiser'].describe()

# advertisers = train['advertiser'].unique()
# advertisers.sort()
# advertisers

In [5]:
advertisers = train['advertiser'].unique()
advertisers.sort()
advertisers

array([1458, 2259, 2261, 2821, 2997, 3358, 3386, 3427, 3476])

In [6]:
# Define data frames

# Train dataframe
train_df = pd.DataFrame(data=[], index=[advertisers], columns=['Imps', 'Clicks', 'Cost', 'avg CPC', 
                                                             'CTR', 'avg CPM', 'eCPC'])
# new_df.loc[3427]['Bids'] = 100

# Validation dataframe
validation_df = pd.DataFrame(data=[], index=[advertisers], columns=['Imps', 'Clicks', 'Cost', 'avg CPC', 
                                                             'CTR', 'avg CPM', 'eCPC'])
# Test dataframe
test_df = pd.DataFrame(data=[], index=[advertisers], columns=['Imps','Clicks','Cost', 'avg CPC', 
                                                             'CTR', 'avg CPM', 'eCPC'])

In [7]:
train.columns

Index(['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [8]:
test.columns

Index(['weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region',
       'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid', 'slotwidth',
       'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative',
       'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [9]:
# advertisers

In [10]:
# ad_group = train.groupby('advertiser')
# ad_group['advertiser'].describe()
# ad_group['advertiser'].count()

In [11]:
# train[train['advertiser'] == 1458].shape

In [12]:
train.columns

Index(['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [13]:
# train.groupby('advertiser')['advertiser'].count()

In [7]:
def dataexploration(dataframe, data):
    advertisers = data.groupby('advertiser')
    dataframe['Imps'] = advertisers['bidid'].nunique()
    dataframe['Clicks'] = advertisers['click'].sum()
    dataframe['Cost'] = advertisers['payprice'].sum()/1000.
    dataframe['CTR'] = (advertisers['click'].sum() / advertisers['bidid'].nunique()) * 100
    dataframe['avg CPC'] = dataframe['Cost'] / dataframe['Clicks']
#     check avg_CPM
    dataframe['avg CPM'] = dataframe['Cost'] / (advertisers['click'].count()) * 1000
    
    # check this with avg cpc?????
    dataframe['eCPC'] = dataframe['Cost'] / dataframe['Clicks']

In [15]:
train_explor = dataexploration(train_df, train)
train_df

,Imps,Clicks,Cost,avg CPC,CTR,avg CPM,eCPC
1458,492353,385,33968.736,88.230483,0.078196,68.992646,88.230483
2259,133673,43,12428.238,289.028791,0.032168,92.974931,289.028791
2261,110122,36,9873.779,274.271639,0.032691,89.662184,274.271639
2821,211366,131,18828.044,143.725527,0.061978,89.077922,143.725527
2997,49829,217,3129.267,14.420585,0.435489,62.800116,14.420585
3358,264956,202,22447.231,111.124906,0.076239,84.720599,111.124906
3386,455041,320,34931.823,109.161947,0.070323,76.766320,109.161947
3427,402806,272,30458.711,111.980555,0.067526,75.616329,111.980555
3476,310835,187,23918.779,127.907909,0.060161,76.950083,127.907909


In [16]:
validation_explor = dataexploration(validation_df, validation)
validation_df

,Imps,Clicks,Cost,avg CPC,CTR,avg CPM,eCPC
1458,62353,49,4294.602,87.644939,0.078585,68.875627,87.644939
2259,16715,2,1568.808,784.404000,0.011965,93.856297,784.404000
2261,13550,3,1214.876,404.958667,0.022140,89.658745,404.958667
2821,26503,23,2394.900,104.126087,0.086783,90.363355,104.126087
2997,6176,26,388.784,14.953231,0.420984,62.950777,14.953231
3358,32939,23,2794.021,121.479174,0.069826,84.824099,121.479174
3386,56665,28,4350.793,155.385464,0.049413,76.780958,155.385464
3427,50183,37,3776.735,102.073919,0.073730,75.259251,102.073919
3476,38841,11,2993.751,272.159182,0.028321,77.077083,272.159182


In [17]:
# Calculate a new data frame for the test data
# test_explor = dataexploration(test_df, test)
# test_df

In [18]:
# import matplotlib.pyplot as plt

# import plotly.plotly as py
# # Learn about API authentication here: https://plot.ly/python/getting-started
# # Find your api_key here: https://plot.ly/settings/api

# y = [3, 10, 7, 5, 3, 4.5, 6, 8.1]
# N = len(y)
# x = range(N)
# width = 1/1.5
# plt.bar(x, y, width, color="blue")


# fig = plt.gcf()
# plot_url = py.plot_mpl(fig, filename='mpl-basic-bar')

In [14]:
# def evaluate_metrics(bidprices):
#     win_items = validation['bidprice'] < bidprices
#     imp = validation[win_items]
#     print('number of clicks', imp['click'].sum())
#     cost = imp['bidprice'].sum() / 1000
#     print('total cost', cost)
#     avg_CTR = (imp['click'].sum() / imp['click'].count()) * 100
# #     Check for CTR
#     print('Click through rate ', avg_CTR)
#     avg_CPC = cost / imp['click'].sum()
#     print('Average CPC', avg_CPC)
#     CPM = (cost) / (imp['click'].count()) * 1000
#     print('Cost per mille', CPM)


def evaluate_metrics(bidprices, display=True):
    win_items = validation['payprice'] < bidprices
    imp = validation[win_items]
    clicks = imp['click'].sum()

    cost = imp['payprice'].sum() / 1000

    avg_CTR = (imp['click'].sum() / imp['click'].count()) * 100
#     Check for CTR

    avg_CPC = cost / imp['click'].sum()
 
    CPM = (cost) / (imp['click'].count()) * 1000
    if display:
        print('number of clicks', clicks)
        print('total cost', cost)
        print('Click through rate ', avg_CTR)
        print('Average CPC', avg_CPC)
        print('Cost per mille', CPM)
    return clicks, cost, avg_CTR

In [21]:
########################################Constant Bidding###############################################################

In [ ]:
avg_bidprice = train['bidprice'].mean()
avg_bidprice

In [13]:
validation = pd.read_csv('we_data/validation.csv')

In [ ]:
validation.columns

In [ ]:
# Calculate metrics for constant bidding
evaluate_metrics(avg_bidprice)

In [ ]:
############################################RANDOM BIDDING ##########################################################

In [ ]:
# Random Bidding 
max_bid = validation['bidprice'].max()
min_bid = validation['bidprice'].min()
print('Max bid is', max_bid)
print('Min bid is', min_bid)

In [ ]:
rand_vec = np.random.uniform(low=min_bid, high=max_bid, size=(validation['bidprice'].count(),))
print('Length of random bid vector is',len(rand_vec))
print('The number of impressions are', validation['bidprice'].count())

In [ ]:
evaluate_metrics(rand_vec)

In [ ]:
#####################################Linear Bidding Strategy###########################################################

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:



# Average CTR (or average click-through rate) is the ratio of ad clicks to impressions in your AdWords campaigns.
# While basic CTR measures the rate of clicks on each ad, average CTR calculates the amount of clicks vs.
# impressions across your campaign, or for each individual keyword.



# lr = linear_model.LinearRegression()
# x = train[['click', 'weekday', 'hour', 'region', 'city','adexchange', 'advertiser', 'bidprice',
#           'slotwidth', 'slotheight', 'slotformat', 'slotprice']]
# x['adexchange'] = pd.to_numeric(x['adexchange'], errors='coerce')
# x['slotformat'] = pd.to_numeric(x['slotformat'], errors='coerce')
# x.fillna(0, inplace=True)
# y = x['bidprice']
# del x['bidprice']
# lr.fit(x,y)




# (x.shape, train.shape)

In [ ]:
#####################################################pCTR##############################################################

In [ ]:
# pd.get_dummies(train['useragent'], prefix='user_agent').head()
# ohe.transform(le.transform(train['useragent'])).shape

In [4]:
train.shape

(2430981, 25)

In [4]:
%%time
def preprocess(data):
    result = data['useragent'].str.split('_')
    data['os'] = np.array(result.tolist())[:,0]
    data['browser'] = np.array(result.tolist())[:,1]
    data['slotsize'] = data['slotwidth'] * data['slotheight']
    result = data['usertag'].str.split(',')
    unique_usertags = {elem
                       for elems in result.tolist()
                       for elem in elems}
    for usertag in unique_usertags:
        data['usertag_'+usertag] = result.apply(lambda x: usertag in x).astype(int)

preprocess(validation)
preprocess(test)
preprocess(train)
# train_sample.shape[0]

CPU times: user 1min 51s, sys: 6.53 s, total: 1min 57s
Wall time: 1min 58s


In [5]:
%%time
train_sample = train.sample(n=int(0.1 * (train.shape[0])))
preprocess(train_sample)
print(train_sample.shape)

(243098, 97)
CPU times: user 12.5 s, sys: 8.75 s, total: 21.2 s
Wall time: 22 s


In [11]:
with pd.option_context('display.max_columns', None):
    display(train_sample.head())

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,os,browser,slotsize,usertag_16753,usertag_11944,usertag_null,usertag_16617,usertag_10684,usertag_10117,usertag_10123,usertag_10063,usertag_11092,usertag_10114,usertag_10140,usertag_10138,usertag_10110,usertag_10048,usertag_10077,usertag_10142,usertag_10116,usertag_13874,usertag_10057,usertag_11680,usertag_14273,usertag_10079,usertag_15398,usertag_11379,usertag_13866,usertag_16706,usertag_10125,usertag_10075,usertag_10133,usertag_10076,usertag_16661,usertag_13776,usertag_13678,usertag_16593,usertag_10126,usertag_10102,usertag_10146,usertag_10145,usertag_10111,usertag_10148,usertag_10127,usertag_13403,usertag_10067,usertag_11576,usertag_10131,usertag_10130,usertag_11632,usertag_10149,usertag_10115,usertag_10083,usertag_16751,usertag_10006,usertag_10052,usertag_10120,usertag_10059,usertag_11512,usertag_13042,usertag_10093,usertag_10031,usertag_10024,usertag_10074,usertag_11423,usertag_10129,usertag_10147,usertag_10118,usertag_13800,usertag_11278,usertag_11724,usertag_13496
2132810,0,4,0,8ba53abc8ba3616d047c2009564dcfea57cfce28,b428f2875e47682e273d0f1d0e8a4849d5eb9152,windows_ie,122.64.20.*,65,65,1,trqRTu1YP5scFsf,d7d131a423a62f44dafd777e3948dae7,null,mm_10075660_3500949_11453278,950,90,0,1,0,cc9b344e950b4f8c2b96537174a343b7,227,48,d29e59bf0f7f8243858b8183f14d4412,3358,"10057,10006",windows,ie,85500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
61849,0,1,16,905f3c265be3c247576a06e1ee9a8e69bd484833,58ed40ae1c47c6e977fcf4a2bd5b36334a4e5a46,windows_ie,113.109.112.*,216,217,3,a05e2e05e42bd2dd5d5c0e182f9dab4e,e995a770896adf3b22e3cb58f487987d,null,Astro_F_Upright,300,250,Na,Na,20,7323,294,160,null,2259,"10059,10077,10006,10111,10127,13403,10063,10116",windows,ie,75000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1160586,0,1,23,2624a9584826ba032001015d07f03f61854a57ce,265655a0a6eff046d7aebed19651a7a2b70b7064,windows_chrome,222.178.220.*,275,275,1,5F97t5E0BTK7XhNrUMpENpn,dedc488b98ca20707bc9a723957e7d1f,null,mm_10027070_118039_10308280,160,600,2,1,0,62f7f9a6dca2f80cc00f17dcda730bc1,227,88,b2e35064f3549d447edbbdfb1f707c8c,3427,"13678,10076,10024,10063,10059,10077,10075,1000...",windows,chrome,96000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1435395,0,0,17,7c65c069215bac2c9c09b066c735a4274f6f163f,c164ce57645345b7083fbea3055432144ce9a982,windows_ie,182.99.50.*,134,136,3,31drTvprdN1RFt,8f048779fa0a0641507345cd8cc63de4,null,Edu_F_Width1,1000,90,0,0,50,832b91d59d0cb5731431653204a76c0e,300,50,bebefa5efe83beee17a3d245e7c5085b,1458,"10024,13403,13866,10057,10083,10052,10075,1006...",windows,ie,90000,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0
1174928,0,4,16,1def0943bdd70d4a743a540aed1d7af0f1eec49c,b8f644bf80dc47810924c5ba0cfdc1d5919f2389,windows_chrome,123.152.128.*,94,96,3,ersbQv1RdoTy1m58uG,3cdc190209dbcbe903d8cbdb99536277,null,ALLINONE_F_Width1,1000,90,0,0,70,86c2543527c86a893d4d4f68810a0416,300,70,43f4a37f42a7c5e6219e2601b26c6976,3386,"13403,13800,10083,10063,10031,10057,10006,1007...",windows,chrome,90000,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0


In [5]:
# with pd.option_context('display.max_rows', None):
#     display(pd.value_counts(train_sample['usertag'].str.split(',').apply(lambda x: list(sorted(x))[0])))

In [6]:
# with pd.option_context('display.max_rows', None):
#     display(pd.value_counts(train_sample['slotvisibility']))

In [12]:
# algo 1
def preprocess_xgb_pctr(data):
    pctr_x = data[['weekday', 'hour', 'advertiser', 
              'slotwidth', 'slotheight', 'slotprice']]
    # region, city, slotformat, adexchange
#     for discrete in ['region', 'city', 'slotformat', 'adexchange']:
    return pctr_x
preprocess_pctr = preprocess_xgb_pctr

In [6]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import gc

# algo 2
def get_train_one_hot_columns(column_name, threshold):
    values = train_sample[column_name]
    counts = pd.value_counts(values)
    mask = values.isin(counts[counts > threshold].index)
    df = pd.get_dummies(values[mask], prefix=column_name).columns
    print(column_name, threshold, df.shape)
    return df

train_one_hot_columns = {}
train_one_hot_columns['slotvisibility'] = get_train_one_hot_columns('slotvisibility', 0)
train_one_hot_columns['slotformat'] = get_train_one_hot_columns('slotformat', 0)
# train_one_hot_columns['region'] = get_train_one_hot_columns('region', 3000)
train_one_hot_columns['region'] = get_train_one_hot_columns('region', 0)
# train_one_hot_columns['city'] = get_train_one_hot_columns('city', 1000)
train_one_hot_columns['city'] = get_train_one_hot_columns('city', 0)
train_one_hot_columns['os'] = get_train_one_hot_columns('os', 0)
train_one_hot_columns['browser'] = get_train_one_hot_columns('browser', 0)
train_one_hot_columns['slotsize'] = get_train_one_hot_columns('slotsize', 0)
train_usertag_columns = [column
                         for column in train_sample.columns
                         if column.startswith('usertag_')]

def ensure_columns(df, column_list):
    for col in np.setdiff1d(column_list, df.columns):
        df[col] = 0
    df = df[column_list]
    return df

def one_hot_encode(data, column):
    train_columns = train_one_hot_columns[column]
    one_hot_data = pd.get_dummies(data[column], prefix=column)
    one_hot_data = ensure_columns(one_hot_data, train_columns)
    return one_hot_data

def preprocess_lg_pctr(data):
    gc.collect()
    pctr_x = data[['weekday', 'hour', 'adexchange', 'advertiser', 
              'slotwidth', 'slotheight']]
    
    pctr_x.loc[:,'adexchange'] = pd.to_numeric(pctr_x['adexchange'], errors='coerce')

    usertag_df = pd.concat([data[column]
                            for column in data.columns
                            if column.startswith('usertag_')],
                          axis=1)
    usertag_df = ensure_columns(usertag_df, train_usertag_columns)

    pctr_x = pd.concat([pctr_x, usertag_df], axis=1)

    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotvisibility')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotformat')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'city')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'region')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'os')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'browser')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotsize')], axis=1)

    # multiply by 1 to make True/False into 1/0
    pctr_x.loc[:, 'floorprice0'] = 1*(data['slotprice'] == 0)
    pctr_x.loc[:, 'floorprice1'] = 1*((1 <= data['slotprice']) & (data['slotprice'] <= 10))
    pctr_x.loc[:, 'floorprice2'] = 1*((11 <= data['slotprice']) & (data['slotprice'] <= 50))
    pctr_x.loc[:, 'floorprice3'] = 1*((51 <= data['slotprice']) & (data['slotprice'] <= 100))
    pctr_x.loc[:, 'floorprice4'] = 1*((101 <= data['slotprice']))

    pctr_x.fillna(0, inplace=True)
    return pctr_x

# gc.collect()
preprocess_pctr = preprocess_lg_pctr

slotvisibility 0 (11,)
slotformat 0 (4,)
region 0 (35,)
city 0 (370,)
os 0 (6,)
browser 0 (9,)
slotsize 0 (26,)


In [277]:
import gc

# algo 3
def freq_ctr_encode(data, column, value=False, one_hot=True):
    ctr_data = train.groupby(column)['click'].mean()
    ctr_data.name = 'ctr'
    freq_data = train.groupby(column)['click'].count()
    freq_data.name = 'freq'
#     ecpc_data = train.groupby(column)['payprice'].sum()/(1000*train_sample.groupby(column)['click'].sum())
#     ecpc_data[~np.isfinite(ecpc_data)] = 0
#     ecpc_data.name = 'ecpc'
    feat_data = pd.concat([ctr_data, freq_data, ecpc_data], axis=1)
    result = data.join(feat_data, on=column, lsuffix='_' + column)
#     result.rename(columns = {'ctr': 'ctr_'+column, 'freq': 'freq_'+column, 'ecpc': 'ecpc_'+column}, inplace=True)
#     result_cols = ['ctr_'+column, 'freq_'+column, 'ecpc_'+column]
    result.rename(columns = {'ctr': 'ctr_'+column, 'freq': 'freq_'+column}, inplace=True)
    result_cols = ['ctr_'+column, 'freq_'+column]
    if not value:
        return result[result_cols]
    if one_hot:
        return pd.concat([result[result_cols], 
                          one_hot_encode(data, column)
                         ], axis=1)
    return result[result_cols + [column]]

def freq_ctr_encode_tag(data, tag):
    mask = train[tag] == 1
    tag_data = train[mask]
    ctr_data = tag_data['click'].mean()
#     freq_data = tag_data['click'].count()
#     ecpc_data = tag_data['payprice'].sum()/(1000*tag_data['click'].sum())
#     if np.any(~np.isfinite(ecpc_data)):
#         ecpc_data = 0
#     result = pd.DataFrame({'ctr_'+tag: data[tag], 'freq_'+tag: data[tag], 'ecpc_'+tag: data[tag]})
    result = pd.DataFrame({'ctr_'+tag: data[tag]})
    result.loc[mask, 'ctr_'+tag] = ctr_data
#     result.loc[mask, 'freq_'+tag] = freq_data
#     result.loc[mask, 'ecpc_'+tag] = ecpc_data
    return result

def preprocess_super_pctr(data):
    gc.collect()
    pctr_x = data[['weekday', 'hour', 'adexchange', 'advertiser','slotwidth', 'slotheight', 'slotprice']]
    pctr_x.loc[:,'adexchange'] = pd.to_numeric(pctr_x['adexchange'], errors='coerce')
    usertag_df = pd.concat([data[column]
                            for column in data.columns
                            if column.startswith('usertag_')],
                          axis=1)
    usertag_df = ensure_columns(usertag_df, train_usertag_columns)

    pctr_x = pd.concat([pctr_x] + [freq_ctr_encode_tag(usertag_df, column)
                                   for column in usertag_df.columns], axis=1)

    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'weekday')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'hour')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'advertiser')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'adexchange')], axis=1)

    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotvisibility')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotformat')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'city')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'region')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'os')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'browser')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotsize')], axis=1)

#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotvisibility')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotformat')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'city')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'region')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'os')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'browser')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotsize')], axis=1)

    # multiply by 1 to make True/False into 1/0
#     train_sample[(train_sample['slotprice'] < 100) & (train_sample['slotprice'] > 0)]['slotprice'].plot.kde()
#     plt.show()

#     for colnum, span in enumerate([(None, 0), (1, 10), (11, 25), (26, 60), (61, 75), (76, 100), (101, None)]):
#         pctr_x.loc[:, 'floorprice' + str(colnum)] = 0
#         if span[0] is None:
#             data_mask = (data['slotprice'] <= span[0])
#             train_mask = (train['slotprice'] <= span[0])
#         elif span[1] is None:
#             data_mask = (data['slotprice'] >= span[1])
#             train_mask = (train['slotprice'] >= span[1])
#         else:
#             data_mask = ((span[0] <= data['slotprice']) & (data['slotprice'] <= span[1]))
#             train_mask = ((span[0] <= train['slotprice']) & (train['slotprice'] <= span[1]))
#         pctr_x.loc[data_mask, 'floorprice0'] = train[train_mask]['click'].mean()

#     pctr_x.loc[:, 'floorprice0'] = 1*(data['slotprice'] <= 0)
#     pctr_x.loc[:, 'floorprice1'] = 1*((1 <= data['slotprice']) & (data['slotprice'] <= 10))
#     pctr_x.loc[:, 'floorprice2'] = 1*((11 <= data['slotprice']) & (data['slotprice'] <= 25))
#     pctr_x.loc[:, 'floorprice3'] = 1*((26 <= data['slotprice']) & (data['slotprice'] <= 60))
#     pctr_x.loc[:, 'floorprice4'] = 1*((61 <= data['slotprice']) & (data['slotprice'] <= 75))
#     pctr_x.loc[:, 'floorprice5'] = 1*((76 <= data['slotprice']) & (data['slotprice'] <= 100))
#     pctr_x.loc[:, 'floorprice6'] = 1*((101 <= data['slotprice']))

    pctr_x.fillna(0, inplace=True)
    return pctr_x

# gc.collect()
preprocess_pctr = preprocess_super_pctr

In [7]:
# CTR: train_sample.groupby('region')['click'].mean()
# frequency: train_sample.groupby('region')['click'].count()
preprocess_pctr

<function __main__.preprocess_lg_pctr>

In [22]:
# result = train_sample['usertag'].str.split(',')
# result = np.array(result.tolist())
# # data['os'] = np.array(result.tolist())[:,0]
# # data['browser'] = np.array(result.tolist())[:,1]

In [82]:
len(train_sample['slotid'].unique())

13973

In [27]:

# bid = base bid×pCT R/avgCT R

In [40]:
# pctr_train_y = train_sample['click']
# pctr_train_x = preprocess_lg_pctr(train_sample)
# lg = linear_model.LogisticRegression()
# lg.fit(pctr_train_x, pctr_train_y)

In [41]:
# pctr_train_x.head()

In [42]:
# logistic_pctr = lg.predict_proba(pctr_train_x)
# avgCTR = train_sample['click'].mean()
# lr = linear_model.LinearRegression(fit_intercept=False)
# lr_feat_x = logistic_pctr[:,1]/avgCTR
# lr_feat_x = lr_feat_x.reshape(-1, 1)
# print (lr_feat_x.shape)
# print(train_sample['bidprice'].shape)
# lr.fit(lr_feat_x, train_sample['bidprice'])

In [43]:



# lr.coef_

In [44]:
# logistic_pctr[:,1]

In [45]:
# # bid = base bid×pCT R/avgCT R

# pctr_valid_x = preprocess_lg_pctr(validation)
# logistic_pctr = lg.predict_proba(pctr_valid_x)
# base_bid = lr.coef_ * 1.2
# logistic_bids = base_bid * logistic_pctr[:,1]/avgCTR
# evaluate_metrics(logistic_bids)

In [46]:
# # bid = base bid×pCT R/avgCT R

# pctr_valid_x = preprocess_lg_pctr(validation)
# logistic_pctr = lg.predict_proba(pctr_valid_x)
# base_bid = lr.coef_ * 1.2
# logistic_bids = base_bid * logistic_pctr[:,1]/avgCTR
# evaluate_metrics(logistic_bids)

In [47]:
# for basebid in range(20,100):
#     sum = 6250 * 1000
#     clicks = 0
#     for pay,click,pCTR in zip(validation["payprice"],validation['click'],logistic_pctr[:,1]):
#         sum_sub = sum - pay
#         bidprice = basebid * pCTR / avgCTR
#         if bidprice-pay > 0 and sum_sub >= 0:
#             sum = sum_sub
#             clicks += click
#     print(basebid, clicks, sum)



In [48]:
# pctr_test_x = preprocess_lg_pctr(test)
# logistic_pctr = lg.predict_proba(pctr_test_x)
# base_bid = lr.coef_ * 1.2

# logistic_bids = base_bid * logistic_pctr[:,1]/avgCTR


In [50]:
# lr.coef_

In [51]:
#####################################################Cv################################################################

In [130]:
# from sklearn.model_selection import cross_val_score


In [131]:
###################################çççççççççççççççççççççççççççç########################################################

In [8]:
%%time

pctr_train_x = preprocess_pctr(train_sample)
assert(np.all(np.isfinite(pctr_train_x)))
print(pctr_train_x.shape)
pctr_train_x.head()

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(243098, 541)
CPU times: user 4.75 s, sys: 4.3 s, total: 9.05 s
Wall time: 9.61 s


In [300]:
# %%time
# import xgboost as xgb

# # algorithm 1:
# # m = xgb.XGBClassifier(max_depth=5, n_estimators=50, learning_rate=0.05, n_jobs=4)
# # algorithm 2:
# m = xgb.XGBClassifier(max_depth=5, n_estimators=100, learning_rate=0.1, n_jobs=4)
# # algorithm 3:
# # auc 80
# # m = linear_model.LogisticRegression(C=0.1, penalty='l1')
# # m = neural_network.MLPClassifier(verbose=True, hidden_layer_sizes=(50, 50))
# # m = xgb.XGBClassifier(max_depth=5, n_estimators=100, learning_rate=0.5, n_jobs=4)

# pctr_train_y = train_sample['click']
# m.fit(pctr_train_x, pctr_train_y)
# xgb_pctr = m.predict_proba(pctr_train_x)

CPU times: user 9min 7s, sys: 9.59 s, total: 9min 17s
Wall time: 2min 40s


In [23]:
%%time
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
pctr_train_y = train_sample['click']
clf.fit(pctr_train_x, pctr_train_y)
rf_pctr = clf.predict_proba(pctr_train_x)

CPU times: user 20 s, sys: 2.7 s, total: 22.7 s
Wall time: 23.1 s


In [24]:
%%time
avgCTR = train_sample['click'].mean()
lr = linear_model.LinearRegression(fit_intercept=False)
lr_feat_x = rf_pctr[:,1]/avgCTR
lr_feat_x = lr_feat_x.reshape(-1, 1)
print (lr_feat_x.shape)
print(train_sample['bidprice'].shape)
lr.fit(lr_feat_x, train_sample['bidprice'])

(243098, 1)
(243098,)
CPU times: user 13.6 ms, sys: 16.1 ms, total: 29.7 ms
Wall time: 26.6 ms


In [303]:
# base_bid = lr.coef_ * 1.85
# print("uhiedh", 80.55 / lr.coef_ )

In [304]:
# pctr_valid_x = preprocess_lg_pctr(validation)
# xgb_valid_pctr = m.predict_proba(pctr_valid_x)
# base_bid = lr.coef_ * 1.84
# xgb_bids = base_bid * xgb_valid_pctr[:,1]/avgCTR
# evaluate_metrics(xgb_bids)

In [11]:
%%time
pctr_valid_x = preprocess_pctr(validation)

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


CPU times: user 5.68 s, sys: 5.42 s, total: 11.1 s
Wall time: 11.3 s


In [25]:
%%time
rf_valid_pctr = clf.predict_proba(pctr_valid_x)

CPU times: user 3.2 s, sys: 2.12 s, total: 5.32 s
Wall time: 5.71 s


In [26]:

from sklearn import metrics
import math

results = pd.DataFrame(columns = ['AUC', 'RMSE'])
results.index.name = 'Advertiser'

for advertiser in validation['advertiser'].unique():
    mask = validation['advertiser'] == advertiser
    adv_pctr_valid_x = pctr_valid_x[mask]
    adv_clicks = validation[mask]['click']
    adv_rf_valid_pctr = clf.predict_proba(adv_pctr_valid_x)[:,1]

    adv_auc = metrics.roc_auc_score(adv_clicks, adv_rf_valid_pctr)
    adv_rmse = math.sqrt(metrics.mean_squared_error(adv_clicks, adv_rf_valid_pctr))

    results.loc[advertiser] = [adv_auc, adv_rmse]

    
all_auc = metrics.roc_auc_score(validation['click'], rf_valid_pctr[:,1])
all_rmse = math.sqrt(metrics.mean_squared_error(validation['click'], rf_valid_pctr[:,1]))

results.loc['All'] = [all_auc, all_rmse]

train_auc = metrics.roc_auc_score(train_sample['click'], rf_pctr[:,1])
train_rmse = math.sqrt(metrics.mean_squared_error(train_sample['click'], rf_pctr[:,1]))

results.loc['Train'] = [train_auc, train_rmse]

results

,AUC,RMSE
Advertiser,,
1458,0.865784,0.027866
3476,0.716689,0.016828
3358,0.805764,0.026325
3427,0.824149,0.027123
3386,0.705552,0.022183
2261,0.868630,0.014887
2821,0.571004,0.029450
2259,0.258063,0.010963
2997,0.567545,0.064771


In [28]:
# rf_pctr
# %%time
budget = 6250
best_scale = None
best_clicks = None

for scale in np.linspace(0.1,10.,1000):
    base_bid = lr.coef_ * scale
    xgb_bids = base_bid * rf_valid_pctr[:,1]/avgCTR
    clicks, cost, avg_CTR = evaluate_metrics(xgb_bids, display=False)
#     print('clicks', clicks, scale, base_bid)
    if best_clicks is None or (clicks > best_clicks and cost < budget):
        best_clicks = clicks
        best_scale = scale
        print('new best clicks: num clicks ', best_clicks, ' scale ', best_scale, ' base bid ', lr.coef_, ' cost ', cost)


new best clicks: num clicks  45  scale  0.1  base bid  [ 115.69422137]  cost  262.784
new best clicks: num clicks  46  scale  0.10990990991  base bid  [ 115.69422137]  cost  305.048
new best clicks: num clicks  47  scale  0.11981981982  base bid  [ 115.69422137]  cost  350.335
new best clicks: num clicks  50  scale  0.12972972973  base bid  [ 115.69422137]  cost  393.855
new best clicks: num clicks  51  scale  0.13963963964  base bid  [ 115.69422137]  cost  440.383
new best clicks: num clicks  53  scale  0.14954954955  base bid  [ 115.69422137]  cost  491.256
new best clicks: num clicks  55  scale  0.159459459459  base bid  [ 115.69422137]  cost  537.975
new best clicks: num clicks  59  scale  0.169369369369  base bid  [ 115.69422137]  cost  586.685
new best clicks: num clicks  61  scale  0.179279279279  base bid  [ 115.69422137]  cost  642.197
new best clicks: num clicks  63  scale  0.189189189189  base bid  [ 115.69422137]  cost  695.589
new best clicks: num clicks  70  scale  0.2090

KeyboardInterrupt: 

In [29]:
print(results.round(3).to_latex())

\begin{tabular}{lrr}
\toprule
{} &    AUC &   RMSE \\
Advertiser &        &        \\
\midrule
1458       &  0.866 &  0.028 \\
3476       &  0.717 &  0.017 \\
3358       &  0.806 &  0.026 \\
3427       &  0.824 &  0.027 \\
3386       &  0.706 &  0.022 \\
2261       &  0.869 &  0.015 \\
2821       &  0.571 &  0.029 \\
2259       &  0.258 &  0.011 \\
2997       &  0.568 &  0.065 \\
All        &  0.796 &  0.026 \\
Train      &  0.925 &  0.028 \\
\bottomrule
\end{tabular}



In [320]:
%%time
pctr_test_x = preprocess_lg_pctr(test)
xgb_test_pctr = m.predict_proba(pctr_test_x)
base_bid = 66.8 * 1.78259319
xgb_bids = base_bid * xgb_test_pctr[:,1]/avgCTR
# evaluate_metrics(xgb_bids)
xgb_bids.shape

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


CPU times: user 13.1 s, sys: 10.6 s, total: 23.6 s
Wall time: 23.7 s


In [ ]:
# csv_df = pd.DataFrame()
# csv_df['bidid'] = test['bidid']
# csv_df['bidprice'] = xgb_bids
# csv_df.to_csv('testing_bidding_price.csv')

In [ ]:
#######################################################################################################################


In [23]:
# test['bidid'].shape

(303375,)

In [1]:
# import xgboost as xgb
# m = xgb.XGBClassifier(max_depth=4, n_estimators=100, n_jobs=2)
# pctr_train_y = train['payprice']
# pctr_train_x = preprocess_lg_pctr(train)
# m.fit(pctr_train_x, pctr_train_y)
# xgb_pctr = m.predict(pctr_train_x)

In [ ]:
# xgb_pctr.head()

In [321]:
csv_df = pd.DataFrame()
csv_df['bidid'] = test['bidid']
csv_df['bidprice'] = xgb_bids
csv_df.to_csv('testing_bidding_price_xglr0402.csv')

In [30]:
# file = pd.read_csv('testing_bidding_price_xglr.csv',  index_col=0)

In [31]:
# file.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,41.693754
1,29167d4caa719788b5a342dbaa25151d53121f80,307.872479
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,78.282445
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,60.343378
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,64.381043


In [322]:
# PSL16u68d1n4
# Linear regression(xgboost for pctr) bidding strategy


!curl http://deepmining.cs.ucl.ac.uk/api/upload/wining_criteria_1/PSL16u68d1n4 -X Post -F 'file=@./testing_bidding_price_xglr0402.csv'


{"ranking": 10, "group": "20", "result": {"impressions": 143694, "cost": 6249.998000002514, "clicks": 163, "ctr": 0.0011343549487104541, "cpc": 38.343546012285366}, "daily submission limit": 3, "today tried times": 3, "best result": {"impressions": 121991, "cost": 6085.894000001979, "clicks": 175, "ctr": 0.0014345320556434491, "cpc": 34.776537142868456}}


In [40]:
#################################################Your Best Bidding Strategy############################################

In [98]:
csv_df = pd.DataFrame()
csv_df['bidid'] = test['bidid']
csv_df['bidprice'] = logistic_bids
csv_df.to_csv('testing_bidding_price.csv')

In [99]:
# PSL16u68d1n4
# Linear regression(logistic regression for pctr) bidding strategy
# !curl http://deepmining.cs.ucl.ac.uk/api/upload/wining_criteria_1/PSL16u68d1n4 -X Post -F 'file=@./testing_bidding_price.csv'


{"ranking": 4, "group": "20", "result": {"impressions": 121286, "cost": 6249.999000001894, "clicks": 90, "ctr": 0.0007420477219134937, "cpc": 69.44443333335438}, "daily submission limit": 3, "today tried times": 2, "best result": {"impressions": 121286, "cost": 6249.999000001894, "clicks": 90, "ctr": 0.0007420477219134937, "cpc": 69.44443333335438}}


In [154]:
# file = pd.read_csv('test_bidprice_20.csv',  index_col=0)
# file.head()